# Segment 1 - Machine Learning Model Mock-up 
### Sam Boester 10/23/2022

In [ ]:
pip install psycopg2

In [8]:
# dependencies

import pandas as pd
import psycopg2 as pg
import plotly.express as px
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:

# Posgres Connection

# engine = pg.connect("dbname='rental_db' user='postgres' host='127.0.0.1' port='5432' password='XXXXXX'")
# df = pd.read_sql('select * from actor', con=engine)

# Confirming that dataframe created.
# df.head()

In [9]:
# Loading Mock CVS, this will be updated to a posgres connection in future segments

# read in CVS


file_path = "resources\Vietnam_USAF_Losses_Cleaned_Data.csv"

usaf_df = pd.read_csv(file_path)

usaf_df.head()




,Crash Date,Crash Time,Aircraft Type,Aircraft S/N,Base,Wing,Squadron,Call Sign,Operations Code,Mission Type,...,Pilot Egress,Pilot Status,Pilot Condition,Pilot Recovered,Co-Pilot/Nav Rank,Co-Pilot/Nav,Co-Pilot/Nav Egress,Co-Pilot/Nav Status,Co-Pilot/Nav Condition,Co-Pilot Recovered
0,620202,_,C-123B,564370,TSN,464TCW,_,_,IC,NaN,...,Crash,KIA,_,_,Capt,Larsen R. D.,Crash,KIA,_,_
1,620211,_,SC-47A,4315732,BHA,_,4400CCTS,_,IC,NaN,...,Crash,KIA,_,_,Capt,Hartson S. G.,Crash,KIA,_,_
2,620613,_,T-28B,0,_,_,_,_,IC,NaN,...,_,Recovered,Uninjured,_,u,_,_,_,_,_
3,620828,_,T-28B,538376,_,_,_,_,IC,NaN,...,_,KIA,_,_,Capt,_,_,_,_,_
4,621015,_,U-10,625909,_,_,_,_,IC,NaN,...,Crash,KIA,_,_,Capt,Foxx R. L.,Crash,KIA,_,_


## Pre-Model Cleaning

In [10]:
# Capture capture variables for model". 

usaf_df_model = usaf_df.filter(['Crash Date','Aircraft Type','Base', 'Defense Type', 'Mission Phase', 'Pilot Status'], axis=1)

usaf_df_model.head()

,Crash Date,Aircraft Type,Base,Defense Type,Mission Phase,Pilot Status
0,620202,C-123B,TSN,_,Enroute,KIA
1,620211,SC-47A,BHA,Gunfire (combat associated),_,KIA
2,620613,T-28B,_,Gunfire,_,Recovered
3,620828,T-28B,_,Gunfire,_,KIA
4,621015,U-10,_,Gunfire,_,KIA


In [14]:
# Remove rows that have string value "_".  For some reason I couldnt chanin the code to do all columns in one line

#update to a loop? like the below sample code?

#for col in cols_to_check:
    #df[col] = df[col].map(lambda x: x.replace(';',''))

usaf_df_modelv3 = usaf_df_model[usaf_df_model["Aircraft Type"].str.contains("_")==False]

usaf_df_modelv4 = usaf_df_modelv3[usaf_df_modelv3["Base"].str.contains("_")==False]

usaf_df_modelv5 = usaf_df_modelv4[usaf_df_modelv4["Defense Type"].str.contains("_")==False]

usaf_df_modelv6 = usaf_df_modelv5[usaf_df_modelv5["Mission Phase"].str.contains("_")==False]

usaf_cleaned_nulls = usaf_df_modelv6[usaf_df_modelv6["Pilot Status"].str.contains("_")==False]

usaf_cleaned_nulls.head(20)

,Crash Date,Aircraft Type,Base,Defense Type,Mission Phase,Pilot Status
24,640324,T-28,BHA,Gunfire,At target,KIA
27,640626,T-28D,BHA,Gunfire,At target,Recovered
28,640806,B-57B,CLK,Gunfire,Enroute,KIA
31,640829,A-1E,BHA,30cal AAA,Enroute,KIA
34,640926,A-1E,BHA,Gunfire,At target,Recovered
35,641002,A-1E,BHA,Gunfire,At target,KIA
37,641026,A-1E,TSN,Gunfire,At target,KIA
40,641119,T-28D,UDN,30cal AAA,Enroute,KIA
41,641119,T-28D,UDN,Gunfire,Enroute,KIA
43,641201,A-1E,BHA,Gunfire,Enroute,Recovered


In [16]:
#Start with dummy coding text variables

usaf_df_dummy = pd.get_dummies(usaf_cleaned_nulls, columns=["Aircraft Type", "Base", "Defense Type", "Mission Phase"])

usaf_df_dummy.head()

,Crash Date,Pilot Status,Aircraft Type_A-1,Aircraft Type_A-1E,Aircraft Type_A-1G,Aircraft Type_A-1H,Aircraft Type_A-1H/J,Aircraft Type_A-1J,Aircraft Type_A-26,Aircraft Type_A-26A,...,Defense Type_Small arms (poss.),Defense Type_Small arms (prob. combat associated),Defense Type_Small arms (prob.),Defense Type_Target debris (combat associated),Mission Phase_At target,Mission Phase_Enroute,Mission Phase_On recce,Mission Phase_On station,Mission Phase_Return,Mission Phase_Transit
24,640324,KIA,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
27,640626,Recovered,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
28,640806,KIA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
31,640829,KIA,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34,640926,Recovered,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [18]:
# Clean Target variable pilot status - Get labels and counts

usaf_df_dummy["Pilot Status"].value_counts()



Recovered                  517
KIA                        348
POW (returned)             156
MIA                        153
POW (died)                   5
POW                          4
KIA (chute failure)          3
Recoverd                     2
u                            2
KIA (chute failed)           1
Recovered DaNang             1
POW (died in captivity)      1
recovered                    1
Name: Pilot Status, dtype: int64

In [20]:
# Clean Target variable pilot status - Create Binomial with dictionary.map

recovered = {'Recovered': 1, 'KIA': 0,'POW (returned)': 1,'MIA': 0,'POW (died)': 0,'POW': 1,'KIA (chute failure)': 0,'Recoverd': 1, 'u': 0, 
    'KIA (chute failed)': 0, 'Recovered DaNang': 1, 'POW (died in captivity)':0, 'Recovered (chute fail)': 1,'recovered': 1,'KIA,body MIA,PJ abandnd': 1, 'POW (died in captivity)': 0}


usaf_df_dummy["Target"] = usaf_df_dummy["Pilot Status"].map(recovered)

usaf_df_dummy["Target"].value_counts

<bound method IndexOpsMixin.value_counts of 24      0
27      1
28      0
31      0
34      1
       ..
1569    0
1571    1
1572    0
1573    1
1574    1
Name: Target, Length: 1194, dtype: int64>

## Random Forest Draft

### During the assignment we will test different supervised models to determine,
### if pilot survival could be predicted by aircraft, artillery, and location data from the Vietnam War. 

In [23]:
# Create feature Variables

Y = usaf_df_dummy["Target"]

x = usaf_df_dummy.drop(["Target", "Pilot Status"], axis=1)

x.head()

,Crash Date,Aircraft Type_A-1,Aircraft Type_A-1E,Aircraft Type_A-1G,Aircraft Type_A-1H,Aircraft Type_A-1H/J,Aircraft Type_A-1J,Aircraft Type_A-26,Aircraft Type_A-26A,Aircraft Type_A-37,...,Defense Type_Small arms (poss.),Defense Type_Small arms (prob. combat associated),Defense Type_Small arms (prob.),Defense Type_Target debris (combat associated),Mission Phase_At target,Mission Phase_Enroute,Mission Phase_On recce,Mission Phase_On station,Mission Phase_Return,Mission Phase_Transit
24,640324,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
27,640626,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
28,640806,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
31,640829,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34,640926,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [24]:
# Check the balance of our target values - We have greater than 1000 and the target of recovered versus non-recovered is roughly proportional
Y.value_counts()

1    681
0    513
Name: Target, dtype: int64

In [25]:
# Splitting into Train and Test sets - modify
X_train, X_test, y_train, y_test = train_test_split(x, Y, random_state=78)

In [26]:
# Resample the training data with the BalancedRandomForestClassifier - may need to change estimator count
# create model instance

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

modle=brfc.fit(X_train, y_train)

In [27]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5931013431013431

In [28]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 61,  56],
       [ 61, 121]], dtype=int64)

In [29]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.50      0.52      0.66      0.51      0.59      0.34       117
          1       0.68      0.66      0.52      0.67      0.59      0.35       182

avg / total       0.61      0.61      0.58      0.61      0.59      0.35       299



# Logistic Reression Draft

#### For a basic logistic regression model we will test different sampling techniques through out the assignment

Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Resample the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids

cluster = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cluster.fit_resample(X_train, y_train)

Counter(y_resampled)

In [ ]:
# Train Model

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

### SMOTEEN

In [ ]:
from imblearn.combine import SMOTEENN

smoteenn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
Counter(y_resampled)

In [ ]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))